In [1]:
from sklearn.ensemble import RandomForestClassifier
from skopt.space import Integer
from skopt.utils import use_named_args
from skimage.feature import local_binary_pattern
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
import csv
from skopt import gp_minimize
import cv2
import concurrent.futures
from os import listdir
from os.path import isfile, join
import numpy as np

path = '../data'
categorias = ['NORMAL', 'PNEUMONIA']

# Espacio de búsqueda para los hiperparámetros de Random Forest
espacio_rf = [
    Integer(10, 200, name='n_estimators'),
    Integer(1, 32, name='max_depth')
]

def procesar_imagen(imagen_path, lbp_params, size, etiqueta):
    imagen = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        print(f'No se pudo leer la imagen: {imagen_path}')
        return None, None

    imagen = cv2.resize(imagen, size)
    lbp = local_binary_pattern(imagen, lbp_params['n_puntos'], lbp_params['radio'], method='uniform')
    # Calcular el histograma de LBP
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, lbp_params['n_puntos'] + 3), range=(0, lbp_params['n_puntos'] + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)

    return hist, etiqueta

def cargar_datos(ruta, lbp_params, size=(128, 128)):
    datos = []
    etiquetas = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i, cat in enumerate(categorias):
            carpeta = join(ruta, cat)
            archivos = [f for f in listdir(carpeta) if isfile(join(carpeta, f))]
            for archivo in archivos:
                imagen_path = join(carpeta, archivo)
                futures.append(executor.submit(procesar_imagen, imagen_path, lbp_params, size, i))

        for future in concurrent.futures.as_completed(futures):
            fd, label = future.result()
            if fd is not None:
                datos.append(fd)
                etiquetas.append(label)

    return np.array(datos), np.array(etiquetas)

# Nombre del archivo CSV para los resultados
nombre_archivo_csv = "../results/models_parameter_combination/rf_combination_with_lbp.csv"
# Inicializar el archivo CSV
with open(nombre_archivo_csv, 'w', newline='') as archivo:
    escritor = csv.writer(archivo)
    escritor.writerow(['n_estimators', 'max_depth', 'Accuracy', 'Recall', 'Specificity', 'Precision', 'F1-Score', 'AUC'])

@use_named_args(espacio_rf)
def funcion_objetivo_rf(n_estimators, max_depth):
    lbp_params = {'radio': 5, 'n_puntos': 18}
    datos_entrenamiento, etiquetas_entrenamiento = cargar_datos(join(path, 'train'), lbp_params)
    datos_prueba, etiquetas_prueba = cargar_datos(join(path, 'test'), lbp_params)

    modelo = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth if max_depth > 0 else None,
        random_state=42,
        n_jobs=-1
    )
    modelo.fit(datos_entrenamiento, etiquetas_entrenamiento)
    etiquetas_predichas = modelo.predict(datos_prueba)

    accuracy = accuracy_score(etiquetas_prueba, etiquetas_predichas)
    f1 = f1_score(etiquetas_prueba, etiquetas_predichas)
    precision = precision_score(etiquetas_prueba, etiquetas_predichas)
    recall = recall_score(etiquetas_prueba, etiquetas_predichas)
    cm = confusion_matrix(etiquetas_prueba, etiquetas_predichas)
    specificity = cm[0,0] / (cm[0,0] + cm[0,1])
    auc = roc_auc_score(etiquetas_prueba, modelo.predict_proba(datos_prueba)[:, 1])
    
    # Registrar en CSV
    with open(nombre_archivo_csv, 'a', newline='') as archivo:
        escritor = csv.writer(archivo)
        escritor.writerow([n_estimators, max_depth, accuracy, recall, specificity, precision, f1, auc])
    
    return -f1

# Ejecutar la optimización
resultado = gp_minimize(funcion_objetivo_rf, espacio_rf, n_calls=20, random_state=42, n_jobs=-1, verbose=True)

# Imprimir el mejor resultado
print(f"Mejor F1-Score: {-resultado.fun}")
print(f"Mejores hiperparámetros: n_estimators={resultado.x[0]}, max_depth={resultado.x[1]}")

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 42.5341
Function value obtained: -0.8284
Current minimum: -0.8284
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 52.8629
Function value obtained: -0.8321
Current minimum: -0.8321
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 43.4205
Function value obtained: -0.8066
Current minimum: -0.8321
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 37.4227
Function value obtained: -0.8311
Current minimum: -0.8321
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 39.3114
Function value obtained: -0.8252
Current minimum: -0.8321
Iteration No: 6 star